In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os
from pathlib import Path
import math

home = Path.home() / 'group' / 'project'

# load the curves

In [ ]:
for i in list((home / 'results' / 'matched_filter' / 'roc_curve').iterdir()): print(i)

In [ ]:
for i in list((home / 'results' / 'beamforming' / 'roc_curve').iterdir()): print(i)

# plot curves for electrons from 1-4 cm and 85-89 deg

In [ ]:
file = '/storage/home/adz6/group/project/results/matched_filter/roc_curve/220223_matched_filter_roc_curves_diff_match_angles85to89_rad1to4.npz'
mf_roc = np.load(file)
file = '/storage/home/adz6/group/project/results/beamforming/roc_curve/220223_time_dep_beamforming_roc_curve_estimate_restricted_params.npz'
bf_roc = np.load(file)

In [ ]:
sns.set_theme(context='talk')
fig = plt.figure(figsize=(13, 8))
cmap = sns.color_palette('mako', n_colors=mf_roc['match'].size )
clist = sns.color_palette('deep', n_colors=10 )
ax = fig.add_subplot(1,1,1)

for i, match in enumerate(mf_roc['match']):
    ax.plot(mf_roc['fpr'], mf_roc['tpr'][i, :], color=cmap[i], label=f'MF, Match={match}')
    
#ax.plot(bf_roc['fpr'], bf_roc['tpr'], color=clist[3], label=f'Time Dep. BF')
ax.plot(np.linspace(0, 1, 501,), np.linspace(0, 1, 501), color='grey', linestyle='--', label='Random Trigger')

ax.set_xscale('log')
ax.set_xlim(1e-8, 1)
ax.legend(loc=2)

ax.set_xlabel('False Positive Rate (Background)')
ax.set_ylabel('True Positive Rate (Efficiency)')
#ax.set_title('ROC Curve Comparison')
ax.set_title('ROC Curve Match Dependence')

plt.tight_layout()
save_path = Path.home()/'group'/'project'/'plots'/'matched_filter'/'roc_curve'/'220223_mf_roc_curve_vs_match_restricted_pitch_and_rad.png'

#plt.savefig(save_path)

# plot curves for electrons from 0-5 cm and 84-90 deg

In [ ]:
file = '/storage/home/adz6/group/project/results/matched_filter/roc_curve/220223_matched_filter_roc_curves_diff_match_angles84to90_rad0to5.npz'
mf_roc = np.load(file)
file = '/storage/home/adz6/group/project/results/beamforming/roc_curve/220223_time_dep_beamforming_roc_curve_estimate.npz'
bf_roc = np.load(file)

In [ ]:
sns.set_theme(context='talk')
fig = plt.figure(figsize=(13, 8))
cmap = sns.color_palette('mako', n_colors=mf_roc['match'].size )
clist = sns.color_palette('deep', n_colors=10 )
ax = fig.add_subplot(1,1,1)

for i, match in enumerate(mf_roc['match']):
    ax.plot(mf_roc['fpr'], mf_roc['tpr'][i, :], color=cmap[i], label=f'MF, Match={match}')
    
#ax.plot(bf_roc['fpr'], bf_roc['tpr'], color=clist[3], label=f'Time Dep. BF')
ax.plot(np.linspace(0, 1, 501,), np.linspace(0, 1, 501), color='grey', linestyle='--', label='Random Trigger')

ax.set_xscale('log')
ax.set_xlim(1e-8, 1)
ax.legend(loc=2)

ax.set_xlabel('False Positive Rate (Background)')
ax.set_ylabel('True Positive Rate (Efficiency)')
#ax.set_title('ROC Curve Comparison')
ax.set_title('ROC Curve Match Dependence')

plt.tight_layout()
save_path = Path.home()/'group'/'project'/'plots'/'matched_filter'/'roc_curve'/'220223_mf_roc_curve_vs_mean_match.png'

#plt.savefig(save_path)



# define the survival functions

In [ ]:
x = np.linspace(0, 12, 201)

noise_sf = stats.rayleigh.sf(x, loc=0, scale=(1/math.sqrt(2)))

In [ ]:
match = 1

signal_sf = stats.rice.sf(
    x.reshape((1, x.size)),
    match * scores.flatten().reshape(scores.size, 1),
    loc = match * scores.flatten().reshape(scores.size, 1) * (1 - 1 / math.sqrt(2)),
    scale=1/math.sqrt(2)
)

In [ ]:
signal_sf.shape

In [ ]:
plt.plot(noise_sf, signal_sf.mean(axis=0))

# select the scores we are most interested in

In [ ]:
r_min = 0.01
r_max = 0.04
pa_min = 85.0
pa_max = 89.0

In [ ]:
pa_grid.shape

In [ ]:
r_select = np.logical_and(
    r_grid.flatten() > r_min,
    r_grid.flatten() < r_max
)

pa_select = np.logical_and(
    pa_grid.flatten() > pa_min,
    pa_grid.flatten() < pa_max
)

select = np.logical_and(
    r_select,
    pa_select
)

In [ ]:
select_signal_sf = signal_sf[select]

In [ ]:
sns.set_theme(context='poster')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)


ax.plot(noise_sf, signal_sf.mean(axis=0), label=r'$\theta\in84-90$ deg and $r\in0-5$ cm' )
ax.plot(noise_sf, select_signal_sf.mean(axis=0),  label=r'$\theta\in85-89$ deg and $r\in1-4$ cm')

ax.set_ylabel('True Positive Rate (Efficiency)')
ax.set_xlabel('False Positive Rate (Background)')
ax.set_xscale('log')
ax.set_xlim(1e-10, 1)
ax.legend(loc=4)

plt.tight_layout()
name = '220214_ideal_mf_roc_curves_selected_parameters'
save_path = home/'plots'/'analysis'/'matched_filter'/'dense_grid'/'score_grid'
save_path.mkdir(parents=True, exist_ok=True)

plt.savefig(str(save_path / name))

# visualize ROC for match assumptions

In [ ]:
sns.set_theme(context='talk')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

match_ratios = [1.0, 0.9, 0.75, 0.5, 0.3, 0.1]

for match in match_ratios:
    signal_sf = stats.rice.sf(
        x.reshape((1, x.size)),
        match * scores.flatten().reshape(scores.size, 1),
        loc = match * scores.flatten().reshape(scores.size, 1) * (1 - 1 / math.sqrt(2)),
        scale=1/math.sqrt(2)
    )
    
    ax.plot(noise_sf, signal_sf[select].mean(axis=0), label=f'Match = {match}')

ax.set_ylabel('True Positive Rate (Efficiency)')
ax.set_xlabel('False Positive Rate (Background)')
ax.set_xscale('log')
ax.set_xlim(1e-10, 1)
ax.legend(loc=2)
#ax.plot(np.linspace(0,1, 10), np.linspace(0,1, 10), color='grey', linestyle='--')


plt.tight_layout()
name = '220215_mf_roc_curves_estimates_match_efficiency_reduction'
save_path = home/'plots'/'analysis'/'matched_filter'/'dense_grid'/'score_grid'
save_path.mkdir(parents=True, exist_ok=True)

plt.savefig(str(save_path / name))

In [ ]:
print(match_ratios)

# visualize MF scores

In [ ]:
sns.set_theme(context='poster', style='ticks')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(scores, aspect='auto', cmap=cmap)
cbar = fig.colorbar(img, label='Score')

In [ ]:
radii = np.unique(r_grid)
angles = np.unique(pa_grid)

r_select = np.logical_and(
    radii.flatten() > r_min,
    radii.flatten() < r_max
)

pa_select = np.logical_and(
    angles.flatten() > pa_min,
    angles.flatten() < pa_max
)

sns.set_theme(context='poster', style='ticks')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(scores[pa_select, :][:, r_select], aspect='auto', cmap=cmap, extent=(r_min, r_max, pa_max, pa_min))
cbar = fig.colorbar(img, label='Score')

ax.set_xlabel('Radial Position (m)')
ax.set_ylabel('Pitch Angle (deg)')

plt.tight_layout()
name = '220214_mf_scores_pitch_angle_and_radius_8192_samples'
save_path = home/'plots'/'analysis'/'matched_filter'/'dense_grid'/'score_grid'
save_path.mkdir(parents=True, exist_ok=True)

plt.savefig(str(save_path / name))


In [ ]:
list((home/'plots'/'analysis'/'matched_filter'/'dense_grid'/).iterdir())

In [ ]:
(home/'plots'/'analysis'/'matched_filter'/'dense_grid'/'score_grid').iterdir()